In [1]:
### Install Lib
!pip install psycopg2

You should consider upgrading via the '/home/lux/.virtualenvs/e-fiscal/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import psycopg2
from requests import request
from collections import Counter
from datetime import datetime

In [3]:
def get_data():
    res = request("GET",
                  "https://api.stackexchange.com/2.2/search?order=desc&sort=activity&intitle=perl&site=stackoverflow")
    res_json = res.json()
    return res_json

In [4]:
a = get_data()

### Obtener el número de respuestas contestadas y no contestadas

In [5]:
answered = Counter((item["is_answered"] for item in a['items']))
print(f"Respuestas Contestadas: {answered[True]}")
print(f"Respuestas No Contestadas: {answered[False]}")

Respuestas Contestadas: 19
Respuestas No Contestadas: 11


### Obtener la respuesta con menor número de vistas

In [9]:
Min = min(a['items'], key=lambda x: x["view_count"])
Min

{'tags': ['excel', 'perl'],
 'owner': {'reputation': 27,
  'user_id': 17757172,
  'user_type': 'registered',
  'profile_image': 'https://www.gravatar.com/avatar/3211187a104ce67f4a41dc0b23646b89?s=256&d=identicon&r=PG',
  'display_name': 'Itsme',
  'link': 'https://stackoverflow.com/users/17757172/itsme'},
 'is_answered': False,
 'view_count': 16,
 'answer_count': 1,
 'score': 0,
 'last_activity_date': 1657132647,
 'creation_date': 1657131264,
 'question_id': 72888192,
 'content_license': 'CC BY-SA 4.0',
 'link': 'https://stackoverflow.com/questions/72888192/usage-of-set-row-in-excel-module-in-perl',
 'title': 'Usage of set_row in Excel module in perl'}

### Obtener la respuesta más vieja y más actual

In [11]:
oldest = min(a['items'], key=lambda x: x["creation_date"])
newest = max(a['items'], key=lambda x: x["creation_date"])

oldest_str = datetime.fromtimestamp(oldest["creation_date"]).strftime("%Y-%m-%dT%H:%M:%S")
newest_str = datetime.fromtimestamp(newest["creation_date"]).strftime("%Y-%m-%dT%H:%M:%S")
print(f"Respuesta mas vieja {oldest_str}")
print(f"Respuesta mas nueva {newest_str}")

Respuesta mas vieja 2012-01-18T20:53:56
Respuesta mas nueva 2022-07-06T16:08:20


### Obtener la respuesta del owner que tenga una mayor reputación

In [12]:
max_reputation = max(a["items"], key=lambda x: x["owner"]["reputation"])
max_reputation

{'tags': ['multidimensional-array', 'slice', 'raku'],
 'owner': {'reputation': 125298,
  'user_id': 2766176,
  'user_type': 'registered',
  'accept_rate': 61,
  'profile_image': 'https://www.gravatar.com/avatar/edd8638efe875601bfe394a8aea5b16d?s=256&d=identicon&r=PG',
  'display_name': 'brian d foy',
  'link': 'https://stackoverflow.com/users/2766176/brian-d-foy'},
 'is_answered': True,
 'view_count': 274,
 'answer_count': 2,
 'score': 9,
 'last_activity_date': 1657116729,
 'creation_date': 1526435514,
 'last_edit_date': 1657116729,
 'question_id': 50361375,
 'content_license': 'CC BY-SA 4.0',
 'link': 'https://stackoverflow.com/questions/50361375/how-can-i-slice-a-shaped-array-in-perl-6',
 'title': 'How can I slice a shaped array in Perl 6?'}

In [92]:
host = os.getenv('POSTGRES_HOST', "localhost")

dbname = os.getenv('POSTGRES_DBNAME', "airports")
port = os.getenv("POSTGRES_PORT", 5432)
user = os.getenv('POSTGRES_USER', "postgres")
password = os.getenv('POSTGRES_PASSWORD', "password")
sslmode = os.getenv('POSTGRES_SSLMODE', "disable")
conn = psycopg2.connect(host=host,
                        port=port,
                        user=user,
                        password=password,
                        dbname=dbname,
                        sslmode=sslmode)
conn.autocommit = True
cursor = conn.cursor()

In [93]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS aerolineas (
ID_AEROLINEA BIGSERIAL PRIMARY KEY,
NOMBRE_AEROLINEA varchar(20)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS aeropuertos (
ID_AEROPUERTO BIGSERIAL PRIMARY KEY,
NOMBRE_AEROPUERTO varchar(20)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS movimientos (
ID_MOVIMIENTO BIGSERIAL PRIMARY KEY,
DESCRIPCION varchar(40)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS vuelos (
ID_AEROLINEA BIGSERIAL,
ID_AEROPUERTO BIGSERIAL,
ID_MOVIMIENTO BIGSERIAL,
DIA date
);
""")



In [94]:
cursor.execute("""
INSERT INTO aerolineas (ID_AEROLINEA, NOMBRE_AEROLINEA)
    VALUES (1, 'Volaris'), (2, 'Aeromar'), (3, 'Interjet'), (4, 'Aeromexico');
""")

cursor.execute("""
INSERT INTO aeropuertos (ID_AEROPUERTO, NOMBRE_AEROPUERTO)
    VALUES (1, 'Benito Juarez'), (2, 'Guanajuato'), (3, 'La paz'), (4, 'Oaxaca');
""")

cursor.execute("""
    INSERT INTO movimientos (ID_MOVIMIENTO, DESCRIPCION)
    VALUES (1, 'Salida'), (2, 'Llegada');
""")

cursor.execute("""
INSERT INTO vuelos (ID_AEROLINEA, ID_AEROPUERTO, ID_MOVIMIENTO, DIA) values (1, 1, 1, '2021-05-02'),
(2, 1, 1, '2021-05-02'),
(3, 2, 2, '2021-05-02'),
(4, 3, 2, '2021-05-02'),
(1, 3, 2, '2021-05-02'),
(2, 1, 1, '2021-05-02'),
(2, 3, 1, '2021-05-04'),
(3, 4, 1, '2021-05-04'),
(3, 4, 1, '2021-05-04')
""")

In [95]:
cursor.execute("""
SELECT nombre_aeropuerto, count(nombre_aeropuerto) as numero_de_eventos
FROM vuelos
         JOIN aeropuertos a on vuelos.id_aeropuerto = a.id_aeropuerto
group by nombre_aeropuerto
having count(nombre_aeropuerto) = (
    SELECT MAX(counts.numero_de_eventos)
    FROM (
             SELECT nombre_aeropuerto, count(nombre_aeropuerto) as numero_de_eventos
             FROM vuelos
                      JOIN aeropuertos a on vuelos.id_aeropuerto = a.id_aeropuerto
             group by nombre_aeropuerto
         ) as counts);
""")

print(f"¿Cuál es el nombre aeropuerto que ha tenido mayor movimiento durante el año?")
for pair in cursor.fetchall():
    print(f"{pair[0]} con {pair[1]} movimientos")

¿Cuál es el nombre aeropuerto que ha tenido mayor movimiento durante el año?
Benito Juarez con 3 movimientos
La paz con 3 movimientos


In [96]:
cursor.execute("""
SELECT nombre_aerolinea, count(nombre_aerolinea) as numero_de_eventos
FROM vuelos
         JOIN aerolineas a2 on vuelos.id_aerolinea = a2.id_aerolinea
group by nombre_aerolinea
having count(nombre_aerolinea) = (
    SELECT MAX(counts.numero_de_eventos)
    FROM (
             SELECT nombre_aerolinea, count(nombre_aerolinea) as numero_de_eventos
            FROM vuelos
                     JOIN aerolineas a2 on vuelos.id_aerolinea = a2.id_aerolinea
            group by nombre_aerolinea
         ) as counts);
""")

print(f"¿Cuál es el nombre aerolínea que ha realizado mayor número de vuelos durante el año?")
for pair in cursor.fetchall():
    print(f"{pair[0]} con {pair[1]} vuelos")

¿Cuál es el nombre aerolínea que ha realizado mayor número de vuelos durante el año?
Aeromar con 3 vuelos
Interjet con 3 vuelos


In [97]:
cursor.execute("""
SELECT dia, count(dia) as vuelos
from vuelos
group by dia
having count(dia) = (
    select max(events.vuelos)
    from (
             SELECT dia, count(dia) as vuelos
             from vuelos
             group by dia
         ) as events
);
""")

print(f"¿En qué día se han tenido mayor número de vuelos?")
for pair in cursor.fetchall():
    print(f"El dia {pair[0]} con {pair[1]} vuelos")

¿En qué día se han tenido mayor número de vuelos?
El dia 2021-05-02 con 6 vuelos


In [98]:
cursor.execute("""
SELECT nombre_aerolinea, min(vuelos)
FROM aerolineas
         JOIN (
    SELECT id_aerolinea, dia, count(dia) as vuelos
    from vuelos
    group by id_aerolinea, dia) as v
              on aerolineas.id_aerolinea = v.id_aerolinea group by nombre_aerolinea having min(vuelos) > 2;
""")
print(f"¿Cuáles son las aerolíneas que tienen mas de 2 vuelos por día?")
req = cursor.fetchall()
for pair in req:
    print(f"La aerolinea {pair[0]} con {pair[1]} vuelos")
if not req:
    print(f"No hay alguna aerolinea que tenga mas de 2 vuelos por dia")

cursor.execute("""
SELECT nombre_aerolinea, min(vuelos)
FROM aerolineas
         JOIN (
    SELECT id_aerolinea, dia, count(dia) as vuelos
    from vuelos
    group by id_aerolinea, dia) as v
              on aerolineas.id_aerolinea = v.id_aerolinea group by nombre_aerolinea having min(vuelos) >= 2;
""")
print(f"¿Cuáles son las aerolíneas que tienen 2 o mas vuelos por día?")
req = cursor.fetchall()
for pair in req:
    print(f"La aerolinea {pair[0]} con {pair[1]} vuelos al dia")
if not req:
    print(f"No hay alguna aerolinea que tenga mas de 2 vuelos por dia")

¿Cuáles son las aerolíneas que tienen mas de 2 vuelos por día?
No hay alguna aerolinea que tenga mas de 2 vuelos por dia
¿Cuáles son las aerolíneas que tienen 2 o mas vuelos por día?
La aerolinea Volaris con 2 vuelos al dia
